## Préparation des données

À cette étape, nous allons charger et préparer les bases de données pour les étapes suivantes du projet que sont la visualisation et le clustering.

Nous pouvons retrouver les bases de données sur le site https://www.data.gouv.fr/fr/ aux adresses suivantes :

* Sites de camping en Ile-de-France : https://www.data.gouv.fr/fr/datasets/carte-des-campings-classes/
* Hôpitaux/Cliniques en France : https://www.data.gouv.fr/fr/datasets/localisation-des-services-daccueil-des-urgences/
* Postes de Police/Gendarmerie en France : https://www.data.gouv.fr/fr/datasets/localisations-des-brigades-de-gendarmeries-et-commissariats-de-police-dans-openstreetmap/
* Carte de Couverture du Réseau Mobile en France : https://www.data.gouv.fr/fr/datasets/mon-reseau-mobile/
* Défibrillateurs en France : https://www.data.gouv.fr/fr/datasets/geodae-base-nationale-des-defibrillateurs/
* Communes en France : https://www.data.gouv.fr/fr/datasets/r/07b7c9a2-d1e2-4da6-9f20-01a7b72d4b12
* Arrondissements de Paris : https://opendata.paris.fr/explore/dataset/arrondissements/download/?format=geojson&timezone=Europe/Berlin&lang=fr


In [1]:
!pip install pandas fiona shapely pyproj rtree
!pip install geopandas

In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [3]:
import requests
import tempfile
import zipfile

temporary_location = tempfile.gettempdir()

#Pour faciliter le téléchargement et le dézippage des données proposées sur data.gouv
def download_unzip(url, dirname = tempfile.gettempdir(), destname = "borders"):
  myfile = requests.get(url)
  open(dirname + '/' + destname + '.zip', 'wb').write(myfile.content)
  with zipfile.ZipFile(dirname + '/' + destname + '.zip', 'r') as zip_ref:
      zip_ref.extractall(dirname + '/' + destname)

Chargement des bases de données :

In [4]:
url1 = "https://www.data.gouv.fr/fr/datasets/r/07b7c9a2-d1e2-4da6-9f20-01a7b72d4b12"
download_unzip(url1)
communes = gpd.read_file(temporary_location + "/borders/communes-20190101.json")

In [5]:
url2 = "https://opendata.paris.fr/explore/dataset/arrondissements/download/?format=geojson&timezone=Europe/Berlin&lang=fr"
arrondissements = gpd.read_file(url2)

In [6]:
url3 = "https://www.data.gouv.fr/fr/datasets/r/322480e7-824b-402d-8dec-ebff968a161d"
camping = gpd.read_file(url3)

In [7]:
url4 = "https://www.data.gouv.fr/fr/datasets/r/6987b47d-7d97-4e3d-8508-ac924c48fc95"
hopitaux = gpd.read_file(url4)

In [8]:
url5 = "https://www.data.gouv.fr/fr/datasets/r/52b2a283-7171-426d-99e1-63efe071cdf0"
police = gpd.read_file(url5)

In [9]:
url6 = "https://www.data.gouv.fr/fr/datasets/r/058154d2-a462-48b7-b726-b77905d16be9"
mobile = gpd.read_file(url6)

In [10]:
url7 = "https://www.data.gouv.fr/fr/datasets/r/86ea48a0-dd94-4a23-b71c-80d3041d7db2"
defibrillateurs = gpd.read_file(url7)

In [11]:
#communes.head()
#camping.head()
#hopitaux.head()
#police.head()
#mobile.head()
#arrondissements.head()
#defibrillateurs.head()

In [12]:
#Pour faire des projections de nos données géographiques sur le CRS EPSG 4326
def proj_4326(x) :
    x.crs = {'init' :'epsg:4326'}
    x = x.to_crs(epsg = 4326)
    return x

### Préparation de communes :

In [13]:
#Stocke les arrondissements de Paris dans la base de données communes 
arrondissements = arrondissements.rename(columns = {"c_arinsee" : "insee"})
arrondissements['insee'] = arrondissements['insee'].astype(str)
communes = communes[~communes.insee.str.startswith("75")].append(arrondissements)
communes["dep"] = communes.insee.str[:2]

In [14]:
#Regroupe les communes en Ile-de-France
communes_idf = communes[communes['dep'].isin(['75', '92', '93', '94', '78', '91', '77', '95'])]

In [15]:
#Nous en tirons seulement les variables susceptibles d'être utiles pour les opérations futures
communes_idf = communes_idf[["insee", "nom", "surf_ha", "geometry", "dep"]]
#Pour avoir des noms de variable plus propres et plus parlants
communes_idf = communes_idf.rename(columns = {"nom" : "Nom", "surf_ha" : "Surface"})

In [16]:
#communes_idf.head()

La base de données communes est maintenant prête pour nos utilisations dans les étapes qui viennent.

### Préparation de departements :

Nous allons utiliser communes pour créer des polygones (et des GeoDataFrame) pour chaque département :

In [17]:
idf_polygon = communes[communes['dep'].isin(['75', '92', '93', '94', '78', '91', '77', '95'])].geometry.unary_union
paris_polygon = communes[communes['dep'].isin(['75'])].geometry.unary_union
essonne_polygon = communes[communes['dep'].isin(['91'])].geometry.unary_union
yvelines_polygon = communes[communes['dep'].isin(['78'])].geometry.unary_union
valdoise_polygon = communes[communes['dep'].isin(['95'])].geometry.unary_union
seineetmarne_polygon = communes[communes['dep'].isin(['77'])].geometry.unary_union
hautsdeseine_polygon = communes[communes['dep'].isin(['92'])].geometry.unary_union
seinesaintdenis_polygon = communes[communes['dep'].isin(['93'])].geometry.unary_union
valdemarne_polygon = communes[communes['dep'].isin(['94'])].geometry.unary_union

idf_polygon_df = gpd.GeoDataFrame(geometry = [idf_polygon])
paris_polygon_df = gpd.GeoDataFrame(geometry = [paris_polygon])
essonne_polygon_df = gpd.GeoDataFrame(geometry = [essonne_polygon])
yvelines_polygon_df = gpd.GeoDataFrame(geometry = [yvelines_polygon])
valdoise_polygon_df = gpd.GeoDataFrame(geometry = [valdoise_polygon])
seineetmarne_polygon_df = gpd.GeoDataFrame(geometry = [seineetmarne_polygon])
hautsdeseine_polygon_df = gpd.GeoDataFrame(geometry = [hautsdeseine_polygon])
seinesaintdenis_polygon_df = gpd.GeoDataFrame(geometry = [seinesaintdenis_polygon])
valdemarne_polygon_df = gpd.GeoDataFrame(geometry = [valdemarne_polygon])

In [18]:
#Regroupe les départements dans une même base de données
frames = [paris_polygon_df, essonne_polygon_df, valdemarne_polygon_df, seineetmarne_polygon_df, yvelines_polygon_df, seinesaintdenis_polygon_df, hautsdeseine_polygon_df, valdoise_polygon_df]
departements_idf = pd.concat(frames)

In [19]:
#Projection sur le CRS EPSG 4326
departements_idf = proj_4326(departements_idf)

/opt/conda/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [20]:
#departements_idf.head()

La base de données departements est prête.

### Préparation de camping :

In [21]:
#Conversion du classement des camping en int
camping["classement"] = camping["classement"].str[:1].astype(int)

Calcul des surfaces des sites de camping :

Les sites classés 3 étoiles ou moins ont au maximum 100 emplacements par hectare et les sites classés 4 étoiles ou plus ont au maximum 80 emplacements par hectare. A partir de cette information, nous pouvons faire une approximation grossière de la surface de chaque site.


In [22]:
surface_3etoiles = camping[camping["classement"] <= 3]["nombre_d_emplacements"]/100
surface_5etoiles = camping[camping["classement"] >= 4]["nombre_d_emplacements"]/80
camping["surface"] = pd.concat([surface_3etoiles, surface_5etoiles])
#Conversion de hectares en m2
camping["surface"] = camping["surface"]*10000 

Création de variables binaires :

Les classements en étoiles nous donnent des informations sur les aménagements dans les sites de camping. 
* Les sites classé 3 étoiles ou plus ont une permanence 24h/24
* Les sites classés 4 étoiles ou plus ont un accès Internet présent partout sur le site
* Les sites classés 2 étoiles ou moins ont un personnel qui parle une langue, ceux classés 3 ou 4 étoiles ont un personnel qui parle 3 langues et les sites classés 5 étoiles ont un personnel qui parle 4 langues

Nous pouvons donc créer des variables binaires pour chaque aménagement (Connexion à Internet partout sur le site, Nombre de langues parlées en accueil, Présence d'une permanence 24h/24). Cela nous sera utile au moment du clustering.


In [23]:
camping["permanence_24h"] = 1*(camping["classement"] >= 3)
camping["langues_accueil"] = 1*(camping["classement"] <= 5) + 2*(camping["classement"] >= 3) + 1*(camping["classement"] == 5)
camping["internet_partout"] = 1*(camping["classement"] >= 4)

In [24]:
#Pour avoir des noms de variable plus propres pour la visualisation
camping = camping.rename(columns = {"nom_commercial" : "Nom", "adresse" : "Adresse", "nombre_d_emplacements" : "Emplacements", "classement" : "Classement", "surface" : "Surface"})

In [25]:
#Projection sur le CRS EPSG 4326
camping = proj_4326(camping)

/opt/conda/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [26]:
#Nous en tirons seulement les variables susceptibles d'être utiles pour les opérations futures
camping = camping[["Adresse", "dep", "Emplacements", "Nom", "Classement", "commune", "code_postal", "geometry", "Surface", "permanence_24h", "langues_accueil", "internet_partout"]]

In [27]:
#camping.head()

La base de données camping est prête.

### Préparation de hopitaux :

In [28]:
#Regroupe les hôpitaux/cliniques en Ile-de-France
hopitaux_idf = hopitaux[hopitaux["geometry"].within(idf_polygon)]
#Pour simplifier la visualisation et le calcul des distances qui vont suivre, 
#nous allons prendre le centroïde au lieu du polygone pour les hôpitaux
hopitaux_idf["geometry"] = hopitaux_idf["geometry"].centroid

/tmp/ipykernel_1037/1818427917.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hopitaux_idf["geometry"] = hopitaux_idf["geometry"].centroid
/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [29]:
#Projection sur le CRS EPSG 4326
hopitaux_idf = proj_4326(hopitaux_idf)

/opt/conda/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [30]:
#Nous en tirons seulement les variables susceptibles d'être utiles pour les opérations futures
hopitaux_idf = hopitaux_idf[["name", "geometry", "addr:housenumber", "addr:street", "addr:postcode", "addr:city"]]
#Pour avoir des noms de variable plus parlants et plus propres pour la visualisation
hopitaux_idf = hopitaux_idf.rename(columns = {"addr:housenumber" : "numero_hab", "addr:street" : "rue", "addr:postcode" : "code_postal", "addr:city" : "ville", "name" : "Nom"})

In [31]:
#hopitaux_idf.head()

La base de données hopitaux est prête.

### Préparation de police :

In [32]:
#Projection sur le CRS EPSG 4326 (coordonnées en mètres avant)
police = police.to_crs(epsg = 4326)

In [33]:
#Regroupe les postes de police/gendarmerie en Ile-de-France
police_idf = police[police['geometry'].within(idf_polygon)]

In [34]:
#Projection sur le CRS EPSG 4326
police_idf = proj_4326(police_idf)

/opt/conda/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [35]:
#Nous en tirons seulement les variables susceptibles d'être utiles pour les opérations futures
police_idf = police_idf[["official_name", "police-FR", "geometry", "addr-housenumber", "addr-street", "addr-postcode", "addr-city", "addr-full"]]
#Pour avoir des noms de variable plus parlants et plus propres pour la visualisation
police_idf = police_idf.rename(columns = {"official_name" : "Nom", "addr-housenumber" : "numero_hab", "addr-street" : "rue", "addr-postcode" : "code_postal", "addr-city" : "ville", "addr-full" : "Adresse"})

In [36]:
#police_idf.head()

La base de données police est prête.

### Préparation de sans_mobile :

In [37]:
#Projection sur le CRS EPSG 4326 (coordonnées en mètres avant)
mobile = mobile.to_crs(epsg = 4326)

In [38]:
#Regroupe les zones avec une bonne couverture de réseau mobile en Ile-de-France
mobile_idf = mobile[mobile["dept"].isin(["75", "77", "78", "91", "92", "93", "94", "95"])]

In [39]:
#Conversion en polygone
mobile_polygon = mobile_idf.geometry.unary_union
#Différence avec le polygone Ile-de-France pour avoir les zones 
#sans une bonne couverture de réseau mobile
sans_mobile_idf = idf_polygon.difference(mobile_polygon)
#Création de GeoDataFrame
sans_mobile_idf_df = gpd.GeoDataFrame(geometry = [sans_mobile_idf])

In [40]:
#Projection sur le CRS EPSG 4326
sans_mobile_idf_df = proj_4326(sans_mobile_idf_df)

/opt/conda/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [41]:
#sans_mobile_idf_df

La base de données sans_mobile est prête.

### Préparation de defibrillateurs :

In [42]:
#Projection sur le CRS EPSG 4326 (coordonnées en mètres avant)
defibrillateurs = defibrillateurs.to_crs(epsg = 4326)

In [43]:
#Regroupe les défibrillateurs en Ile-de-France
defibrillateurs_idf = defibrillateurs[defibrillateurs["geometry"].within(idf_polygon)]
defibrillateurs_idf["geometry"] = defibrillateurs_idf["geometry"].centroid

/tmp/ipykernel_1037/3660121001.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  defibrillateurs_idf["geometry"] = defibrillateurs_idf["geometry"].centroid
/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [44]:
#Projection sur le CRS EPSG 4326
defibrillateurs_idf = proj_4326(defibrillateurs_idf)

/opt/conda/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [45]:
#Nous en tirons seulement les variables susceptibles d'être utiles pour les opérations futures
defibrillateurs_idf = defibrillateurs_idf[["c_adr_num", "c_adr_voie", "c_com_cp", "c_com_insee", "c_com_nom", "geometry"]]
#Pour avoir des noms de variable plus parlants
defibrillateurs_idf = defibrillateurs_idf.rename(columns = {"c_adr_num" : "numero_hab", "c_adr_voie" : "rue", "c_com_cp" : "code_postal", "c_com_nom" : "commune", "c_com_insee" : "insee"})

In [46]:
#defibrillateurs_idf.head()

La base de données defibrillateurs est prête.

À ce stade, nos bases de données telles qu'elles suffisent pour une première visualisation des sites de camping et les alentours (hôpitaux, zones sans couverture mobile, etc.). En revanche, pour le clustering (des sites de camping), utiliser uniquement la localisation de ces éléments ne suffit pas. Nous avons besoin de créer des variables supplémentaires qui demandent plus de manipulations.

### Préparation pour le clustering :

La première variable que nous allons créer est le rapport entre la surface sans réseau de chaque site de camping et sa surface totale. 

Puisque nous n'avons que des points qui représentent chaque site de camping, nous allons supposer que chaque site est un cercle.

In [47]:
#Calcul du rayon du cercle/site en utilisant la surface
rayon_camping = np.sqrt((camping["Surface"]/np.pi)) + 5

In [48]:
cercle_camping = camping
#Projection sur le CRS EPSG 4326
cercle_camping = proj_4326(cercle_camping)

/opt/conda/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


Création de cercles autour du point représentatif du site de camping :

Ici, la fonction buffer prend des distances en degrés, donc nous allons convertir la distance en degrés en divisant par 111000 car 1° = 111000m

In [49]:
cercle_camping = cercle_camping.buffer(rayon_camping/111000)

/tmp/ipykernel_1037/2270548132.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cercle_camping = cercle_camping.buffer(rayon_camping/111000)


In [50]:
#Création de GeoDataFrame
cercle_camping = gpd.GeoDataFrame(cercle_camping, geometry = cercle_camping.values)
#Projection sur le CRS EPSG 4326
cercle_camping = proj_4326(cercle_camping)

/opt/conda/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [51]:
cercle_camping[cercle_camping["geometry"].intersects(sans_mobile_idf) == True]

,0,geometry
27,"POLYGON ((2.43550 48.25663, 2.43550 48.25657, ...","POLYGON ((2.43550 48.25663, 2.43550 48.25657, ..."


Ici, nous pouvons voir que seulement un site de camping se chevauche avec les zones sans réseau. Ce n'est donc pas un élément qui permet de discriminer entre les sites de camping. On ne va pas l'utiliser pour le clustering.

Terminons quand même la tâche :

In [52]:
#Projection sur le CRS EPSG 3857 pour avoir des coordonnées en mètres
cercle_camping = cercle_camping.to_crs(epsg = 3857)
#Calcul de l'aire du site
cercle_camping["Aire"] = cercle_camping.area

In [53]:
#Intersection avec le polygone des zones sans réseau pour trouver 
#la surface sans réseau de chaque site de camping
overlap = cercle_camping.intersection(sans_mobile_idf)
#Création de GeoDataFrame
overlap = gpd.GeoDataFrame(overlap, geometry = overlap.values)
#Projection sur le CRS EPSG 3857 pour avoir des coordonnées en mètres
overlap = overlap.to_crs(epsg = 3857)
#Calcul de l'aire du site sans réseau
overlap["Aire"] = overlap.area

In [54]:
#Calcul du rapport
overlap["rapport"] = overlap["Aire"]/cercle_camping["Aire"]

In [55]:
camping["rapport_sans_mobile"] = overlap["rapport"]

En vue du clustering, nous allons aussi :

1. Calculer la distance entre les sites de camping et les postes de police/hôpitaux 

2. Trouver les sites de camping qui ont accès à un défibrillateur dans un rayon de 2km


In [57]:
!pip install osmnx

     |████████████████████████████████| 95 kB 937 kB/s            
     |████████████████████████████████| 16.8 MB 542 kB/s            
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.22.0 which is incompatible.


In [58]:
import networkx as nx
import osmnx as ox

%matplotlib inline
ox.__version__

'1.1.2'

In [59]:
#Chargement des routes en Ile-de-France
G = ox.graph_from_place("ile-de-france, France", network_type="drive")
#fig, ax = ox.plot_graph(G)

1. Calcul des distances :

In [61]:
pd.set_option('display.max_rows', None)

In [62]:
#Pour voir la forme des coordonnées (en string)
#print(str(camping["geometry"]))

In [63]:
#Sépare les différentes coordonnées individuelles (en string) 
#et les met dans une liste
coordinates_camping = str(camping["geometry"]).split("\n")
coordinates_camping = coordinates_camping[:-1]
coordinates_police = str(police_idf["geometry"]).split("\n")
coordinates_police = coordinates_police[:-1]

In [66]:
#Pour convertir les coordonnées (en string) de la forme Point (x y) 
#en une liste de float facilement manipulable
def get_coordinates(test) :
    coordinates = []
    for i in test:
        j = i.split('(') #Sépare "Point" de x y)
        k = j[-1][:-1].split(" ") #Sépare x et y
        coordinates.append([float(k[0]), float(k[1])]) #Convertit x et y en float et les stocke dans une liste
    return(coordinates)

In [67]:
#Convertit les coordonnées des sites de camping en une liste de coordonnées
coordinates_camping_xy = get_coordinates(coordinates_camping)
#coordinates_camping_xy

In [68]:
#Convertit les coordonnées des postes de police en une liste de coordonnées
coordinates_police_xy = get_coordinates(coordinates_police)
#coordinates_police_xy

In [69]:
#Pour retrouver, parmi une liste d'éléments (de postes de police/hôpitaux), 
#celui qui est le plus proche (son indice) d'un endroit 
#donné (un site de camping donné), en utilisant kNN
def get_index(coordinates_police_xy, coordinates_camping_xy) :
    i = coordinates_camping_xy #Pour un endroit donné
    neigh = NearestNeighbors(n_neighbors = 1) #Nous cherchons le plus proche
    neigh.fit(coordinates_police_xy) #Parmi une liste
    #Nous retrouvons l'indice du plus proche
    nearest = neigh.kneighbors([i])
    return(nearest[1][0][0])


In [70]:
#Pour retrouver la distance entre un endroit (un site de camping donné) et 
#l'élément (le poste de police/hôpital) le plus proche
def get_distance(start, end) :
    distance = []
    i = start
    dist = []
    j = end[get_index(end, start)] #Indice de l'élément le plus proche
    orig = ox.distance.nearest_nodes(G, X = i[0], Y = i[1]) #Route la plus proche de l'origine
    dest = ox.distance.nearest_nodes(G, X = j[0] , Y = j[1]) #Route la plus proche de la destination
    route = ox.shortest_path(G, orig, dest, weight = "travel_time") #Calcul du trajet le plus rapide entre origine et destination
    #fig, ax = ox.plot_graph_route(G, route, node_size=0)
    #Calcul de la distance du trajet le plus rapide
    edge_lengths = ox.utils_graph.get_route_edge_attributes(G, route, "length") 
    dist.append(round(sum(edge_lengths)))
    distance = min(dist)
    return(distance)

In [71]:
from sklearn.neighbors import NearestNeighbors

Calcul de la distance entre chaque site de camping et le poste de police le plus proche :

In [72]:
distance_police = []
index = 0
for i in coordinates_camping_xy : #Pour chaque site de camping
    index = index + 1
    dist = get_distance(i, coordinates_police_xy) #Calcul de la distance au poste de police le plus proche
    distance_police.append(dist) #Regroupe dans une liste
    print(index, "/", len(coordinates_camping_xy), " finished")
camping["distance_police"] = distance_police #Création de la variable distance_police

1 / 65  finished
2 / 65  finished
3 / 65  finished
4 / 65  finished
5 / 65  finished
6 / 65  finished
7 / 65  finished
8 / 65  finished
9 / 65  finished
10 / 65  finished
11 / 65  finished
12 / 65  finished
13 / 65  finished
14 / 65  finished
15 / 65  finished
16 / 65  finished
17 / 65  finished
18 / 65  finished
19 / 65  finished
20 / 65  finished
21 / 65  finished
22 / 65  finished
23 / 65  finished
24 / 65  finished
25 / 65  finished
26 / 65  finished
27 / 65  finished
28 / 65  finished
29 / 65  finished
30 / 65  finished
31 / 65  finished
32 / 65  finished
33 / 65  finished
34 / 65  finished
35 / 65  finished
36 / 65  finished
37 / 65  finished
38 / 65  finished
39 / 65  finished
40 / 65  finished
41 / 65  finished
42 / 65  finished
43 / 65  finished
44 / 65  finished
45 / 65  finished
46 / 65  finished
47 / 65  finished
48 / 65  finished
49 / 65  finished
50 / 65  finished
51 / 65  finished
52 / 65  finished
53 / 65  finished
54 / 65  finished
55 / 65  finished
56 / 65  finished
5

In [73]:
#Sépare les différentes coordonnées individuelles (en string) et 
#les met dans une liste
coordinates_hopitaux = str(hopitaux_idf["geometry"]).split("\n")
coordinates_hopitaux = coordinates_hopitaux[:-1]
#Convertit les coordonnées des hôpitaux en une liste de coordonnées
coordinates_hopitaux_xy = get_coordinates(coordinates_hopitaux)

Calcul de la distance entre chaque site de camping et l'hôpital le plus proche :

In [74]:
distance_hopitaux = []
index = 0
for i in coordinates_camping_xy : #Pour chaque site de camping
    index = index + 1
    dist = get_distance(i, coordinates_hopitaux_xy) #Calcul de la distance à l'hôpital le plus proche
    distance_hopitaux.append(dist) #Regroupe dans une liste
    print(index, "/", len(coordinates_camping_xy), " finished")
camping["distance_hopitaux"] = distance_hopitaux #Création de la variable distance_hôpitaux

1 / 65  finished
2 / 65  finished
3 / 65  finished
4 / 65  finished
5 / 65  finished
6 / 65  finished
7 / 65  finished
8 / 65  finished
9 / 65  finished
10 / 65  finished
11 / 65  finished
12 / 65  finished
13 / 65  finished
14 / 65  finished
15 / 65  finished
16 / 65  finished
17 / 65  finished
18 / 65  finished
19 / 65  finished
20 / 65  finished
21 / 65  finished
22 / 65  finished
23 / 65  finished
24 / 65  finished
25 / 65  finished
26 / 65  finished
27 / 65  finished
28 / 65  finished
29 / 65  finished
30 / 65  finished
31 / 65  finished
32 / 65  finished
33 / 65  finished
34 / 65  finished
35 / 65  finished
36 / 65  finished
37 / 65  finished
38 / 65  finished
39 / 65  finished
40 / 65  finished
41 / 65  finished
42 / 65  finished
43 / 65  finished
44 / 65  finished
45 / 65  finished
46 / 65  finished
47 / 65  finished
48 / 65  finished
49 / 65  finished
50 / 65  finished
51 / 65  finished
52 / 65  finished
53 / 65  finished
54 / 65  finished
55 / 65  finished
56 / 65  finished
5

2. Trouver les sites de camping qui ont accès à un défibrillateur dans un rayon de 2km

In [75]:
#Sépare les différentes coordonnées individuelles (en string) 
#et les met dans une liste
coordinates_defibrillateurs = str(defibrillateurs_idf["geometry"]).split("\n")
coordinates_defibrillateurs = coordinates_defibrillateurs[:-1]
#Convertit les coordonnées des défibrillateurs en une liste de coordonnées
coordinates_defibrillateurs_xy = get_coordinates(coordinates_defibrillateurs)

Nous retrouvons les sites de camping qui ont accès à un défibrillateur dans un rayon de 2km :

In [77]:
distance_defibrillateurs = []
index = 0
for i in coordinates_camping_xy : #Pour chaque site de camping
    index = index + 1
    dist = get_distance(i, coordinates_defibrillateurs_xy) #Calcul de la distance au défibrillateur le plus proche
    num = 1*(dist < 2000) #Vérifie si le défibrillateur est à moins de 2km
    distance_defibrillateurs.append(num) #Regroupe dans une liste
    print(index, "/", len(coordinates_camping_xy), " finished")
camping["defib_moins_2km"] = distance_defibrillateurs #Création de la variable defib_moins_2km

1 / 65  finished
2 / 65  finished
3 / 65  finished
4 / 65  finished
5 / 65  finished
6 / 65  finished
7 / 65  finished
8 / 65  finished
9 / 65  finished
10 / 65  finished
11 / 65  finished
12 / 65  finished
13 / 65  finished
14 / 65  finished
15 / 65  finished
16 / 65  finished
17 / 65  finished
18 / 65  finished
19 / 65  finished
20 / 65  finished
21 / 65  finished
22 / 65  finished
23 / 65  finished
24 / 65  finished
25 / 65  finished
26 / 65  finished
27 / 65  finished
28 / 65  finished
29 / 65  finished
30 / 65  finished
31 / 65  finished
32 / 65  finished
33 / 65  finished
34 / 65  finished
35 / 65  finished
36 / 65  finished
37 / 65  finished
38 / 65  finished
39 / 65  finished
40 / 65  finished
41 / 65  finished
42 / 65  finished
43 / 65  finished
44 / 65  finished
45 / 65  finished
46 / 65  finished
47 / 65  finished
48 / 65  finished
49 / 65  finished
50 / 65  finished
51 / 65  finished
52 / 65  finished
53 / 65  finished
54 / 65  finished
55 / 65  finished
56 / 65  finished
5

In [78]:
#camping

,Unnamed: 0,Adresse,dep,Emplacements,Nom,Classement,commune,code_postal,geometry,Surface,permanence_24h,langues_accueil,internet_partout,rapport_sans_mobile,distance_police,distance_hopitaux,defib_moins_2km
0,0,Route du Château d'eau,78,180,CAMPING HUTTOPIA RAMBOUILLET,3,RAMBOUILLET,78120,POINT (1.835694 48.630059),18000.0,1,3,0,0.0,2817,6807,0
1,1,11 CHEMIN DU CAMPING,77,440,CAMPING LES PRÉS DE LA FONTAINE,3,HERMÉ,77114,POINT (3.338549 48.472784),44000.0,1,3,0,0.0,4953,10485,1
2,2,26 Route de Vauvert,91,230,CARAVANING LE VAUVERT,4,ORMOY-LA-RIVIÈRE,91150,POINT (2.143939 48.411278),28750.0,1,3,1,0.0,5177,691,1
3,3,chemin des ponceaux,77,140,CAMPING LES ACACIAS,2,BETON-BAZOCHES,77320,POINT (3.239464 48.707003),14000.0,0,1,0,0.0,16466,23260,0
4,4,24 Place de la gare de Faremoutiers,77,316,LE CHÊNE GRIS,4,POMMEUSE,77515,POINT (2.99998 48.800512),39500.0,1,3,1,0.0,8711,12013,1
5,5,Le Clos Gouget,78,100,CARAVANIG CLUB DU CLOS GOUGET,2,BOINVILLIERS,78200,POINT (1.661974 48.931593),10000.0,0,1,0,0.0,9188,10804,0
6,6,49 Rue des Bondons,77,247,CARAVANING DES BONDONS,4,LA FERTÉ-SOUS-JOUARRE,77260,POINT (3.150584 48.948976),30875.0,1,3,1,0.0,2139,23095,0
7,7,4 rue de l'ancienne église,77,44,BLP CARAVANING,1,PRÉCY-SUR-MARNE,77410,POINT (2.776459 48.930786),4400.0,0,1,0,0.0,15316,19206,0
8,8,ROUTE DE LA PETITE BEAUCE,91,138,HÉLIOMONDE,3,SAINT-CHÉRON,91530,POINT (2.12983 48.53845),13800.0,1,3,0,0.0,2703,30123,1
9,9,rue de montereau,77,84,CAMPING LE REFUGE,2,GRANDPUITS-BAILLY-CARROIS,77720,POINT (2.970295 48.583675),8400.0,0,1,0,0.0,7179,38519,0


In [ ]:
orig = ox.distance.nearest_nodes(G, X=2.27532, Y=48.86756)
dest = ox.distance.nearest_nodes(G, X=2.23466 , Y=48.86822)
route = ox.shortest_path(G, orig, dest, weight="travel_time")
fig, ax = ox.plot_graph_route(G, route, node_size=0)
edge_lengths = ox.utils_graph.get_route_edge_attributes(G, route, "length")
dist = round(sum(edge_lengths))

In [ ]:
#Conversion en csv des bases de données préparées pour pouvoir les utiliser pour la visualisation et le clustering
police_idf.to_csv("police.csv")
hopitaux_idf.to_csv("hopitaux.csv")
camping.to_csv("camping.csv")
defibrillateurs_idf.to_csv("defibrillateur.csv")
sans_mobile_idf_df.to_csv("sans_mobile.csv")
departements_idf.to_csv("departements.csv")
communes_idf.to_csv("communes.csv")